## Hit song Predection

## Introduction

- This is project for Misk Skills - Data Science Immersive 2021-02. 
- The goal is to build a model to predit what make a song a hit song.
- Using Billboerd hot 100 weekly list.


#### About the dataset 

- The dataset containt weekly charts form [Billboard](https://www.billboard.com) for the [The Hot 100](https://www.billboard.com/charts/hot-100/) list. 
- Downloaded from Kaggle. [Source](https://www.kaggle.com/dhruvildave/billboard-the-hot-100-songs)
- The dataset contain the weekly lists from 1958 to 2021. 


### Dictionary 

- date:   
        - the chart week. 
- rank:  
the rank of the song on that week.
- song:  
Name of the song.
- artist:  
Artist including featured artists
- last-week:  
The rank of the song on the previos week.   
Null mean the song was not on the chart last week or its it frist time entring the hot 100.
- peak-rank:   
The highest rank the song got on the chart on it life time. throuout all the weeks.
- weeks-on-board:   
Number of weeks the song has been on the bored 

## First Look at the data

In [56]:
#Importing libraries 
import pandas as pd
import re

In [57]:
# Initial data set
df =pd.read_csv("../data/original_hot_100.csv")

In [58]:
#quick look at the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


In [59]:
#first 5 observation on the dataset
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [60]:
#how many observation and features on the data set
df.shape

(330087, 7)

## Exploring the Original Dataset

In [61]:
#the years of the weekly charts 
print(pd.DatetimeIndex(df['date']).year.unique())
print("total number of years :" ) 
print( pd.DatetimeIndex(df['date']).year.nunique())

Int64Index([2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011,
            2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000,
            1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989,
            1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978,
            1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967,
            1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958],
           dtype='int64', name='date')
total number of years :
64


**64** years, weekly charts = **12**  * **4** = **48** charts a year.        
**48** * **64** = **3072** total charts.

In [62]:
#Number of unique songs and how many time a song has been on the bored
df[["song","artist"]].value_counts()

song                      artist                      
Blinding Lights           The Weeknd                      90
Radioactive               Imagine Dragons                 87
Sail                      AWOLNATION                      79
I'm Yours                 Jason Mraz                      76
How Do I Live             LeAnn Rimes                     69
                                                          ..
I Won't Give Up           Jana Kramer                      1
I Won't Back Down         Blake Shelton & Dia Frampton     1
I Will Always Love You    Glee Cast                        1
I Who Have Nothing        Jordin Sparks                    1
Trap This Way (This Way)  Lil Uzi Vert                     1
Length: 29681, dtype: int64

In [63]:
#Number of unique artists 
df.artist.nunique()

10205

## Data prepering for use on Spotify API

The goal of using Spotify API is to pull songs information form it.     
Since there are *330087* and some of them are duplicate song, because the same song could be on the charts for multiple times.  
Im only intrested in a list of unique songs to run on the soptify Api and pull its informaton once.


#### First: Getting a subset for songs and artists 

In [64]:
# getting a dataframe of only the artist and the song 
song_artist_Only = df[["song","artist"]]

In [65]:
song_artist_Only.head()

,song,artist
0,Easy On Me,Adele
1,Stay,The Kid LAROI & Justin Bieber
2,Industry Baby,Lil Nas X & Jack Harlow
3,Fancy Like,Walker Hayes
4,Bad Habits,Ed Sheeran


In [66]:
song_artist_Only.shape

(330087, 2)

#### Second: Removing duplicate entries form the dataframe

In [67]:
# removing duplicate songs on the list 
# using the song and the artist as an index since some songs have same name but diffrent artist

song_artist_Only["is_duplicate"] = song_artist_Only.duplicated(subset=["song","artist"])

/var/folders/5n/qk5gndxn7k19r8b5gd8_w74w0000gn/T/ipykernel_1337/2793131396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_artist_Only["is_duplicate"] = song_artist_Only.duplicated(subset=["song","artist"])


In [68]:
song_artist_Only.head()

,song,artist,is_duplicate
0,Easy On Me,Adele,False
1,Stay,The Kid LAROI & Justin Bieber,False
2,Industry Baby,Lil Nas X & Jack Harlow,False
3,Fancy Like,Walker Hayes,False
4,Bad Habits,Ed Sheeran,False


In [69]:
# saveing the subset ON NEW OBJECT
unique_songs_only = song_artist_Only[song_artist_Only["is_duplicate"] == False].reset_index()

In [70]:
# dropping the is_duplicate column
unique_songs_only.drop(columns=["is_duplicate"], inplace=True)

In [71]:
# Keeping the original index in case needed for merging later 
unique_songs_only.rename(columns={"index" : "original_index"}, inplace=True)

In [72]:
unique_songs_only.head()

,original_index,song,artist
0,0,Easy On Me,Adele
1,1,Stay,The Kid LAROI & Justin Bieber
2,2,Industry Baby,Lil Nas X & Jack Harlow
3,3,Fancy Like,Walker Hayes
4,4,Bad Habits,Ed Sheeran


#### Third: Splitting the artist column to 3 diffrent objects 

- Primary Artist, featured_artist_1 and featured_artist_2
- I need only the main artist name to run on spotify api when i do my search.

In [73]:
# a function that take the artist value and split it into 3 
def artists_split(artists):
    result = [x.strip() for x in re.split(r'&|Featuring',artists)]
    return result
    

In [74]:
# applying the function
artists_separated = unique_songs_only["artist"].apply(artists_split)

In [75]:
# 3 features added to the dataframe
unique_songs_only["primary_artist"] = None
unique_songs_only["featured_artist_1"] = None
unique_songs_only["featured_artist_2"] = None

In [76]:
# for loop to take the values in the artist column after splitting and assign them each to thier appropriate column  
for x in range(len(artists_separated)):
        unique_songs_only["primary_artist"][x] = artists_separated[x][0]
        if len(artists_separated[x]) > 1:
            unique_songs_only["featured_artist_1"][x] = artists_separated[x][1]
        if len(artists_separated[x]) > 2:
            unique_songs_only["featured_artist_2"][x] = artists_separated[x][2]
        

/var/folders/5n/qk5gndxn7k19r8b5gd8_w74w0000gn/T/ipykernel_1337/95286634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_songs_only["primary_artist"][x] = artists_separated[x][0]
/var/folders/5n/qk5gndxn7k19r8b5gd8_w74w0000gn/T/ipykernel_1337/95286634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_songs_only["featured_artist_1"][x] = artists_separated[x][1]
/var/folders/5n/qk5gndxn7k19r8b5gd8_w74w0000gn/T/ipykernel_1337/95286634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [77]:
unique_songs_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29681 entries, 0 to 29680
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_index     29681 non-null  int64 
 1   song               29681 non-null  object
 2   artist             29681 non-null  object
 3   primary_artist     29681 non-null  object
 4   featured_artist_1  4367 non-null   object
 5   featured_artist_2  592 non-null    object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


#### Finally: Saveing the dataframe as csv file

In [78]:
# saving the data to csv file 
#unique_songs_only.to_csv("../data/unique_songs.csv")